In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import LinearSegmentedColormap, Normalize  # Import Normalize here

# Crear mapas de colores personalizados
def create_custom_cmap():
    # Rayas gusano: fondo negro y concentraciones verdes
    rayas_gusano_cmap = LinearSegmentedColormap.from_list('rayas_gusano', ['#000000', '#00FF00'])
    
    # Cerebro coral: fondo negro y concentraciones rosa grisáceo claro
    cerebro_coral_cmap = LinearSegmentedColormap.from_list('cerebro_coral', [
        '#000000',  # Negro (fondo)
        '#2E2E2E',  # Gris oscuro
        '#6A5ACD',  # Azul lavanda (transición)
        '#FFC0CB',  # Rosa claro
    ])
    
    return {
        "cerebro_coral": {"F": 0.082, "K": 0.060, "cmap": cerebro_coral_cmap},
        "rayas_gusano": {"F": 0.058, "K": 0.065, "cmap": rayas_gusano_cmap},
        "leopardo": {"F": 0.034, "K": 0.0618, "cmap": "YlOrBr"},  # Amarillo y marrón para leopardo
        "laberintos": {"F": 0.030, "K": 0.0565, "cmap": "binary"},  # Blanco y negro para laberintos
    }

# Obtener los parámetros con los mapas de colores personalizados
PATTERNS = create_custom_cmap()

class ReactionDiffusion:
    def __init__(self, size=512, F=0.03, K=0.0565, cmap="inferno"):
        self.size = size
        self.F = F
        self.K = K
        self.dA = 1.0
        self.dB = 0.5
        self.dt = 0.1  # Paso de tiempo más pequeño para mayor precisión
        
        # Inicializar concentraciones
        self.U, self.V = self.initialize_chemicals()
        
        # Configurar visualización
        self.fig, self.ax = plt.subplots(figsize=(8, 8), dpi=150)
        self.img = self.ax.imshow(
            self.V, 
            cmap=cmap,  # Mapa de colores personalizado
            vmin=0, 
            vmax=1,
            interpolation='bicubic'  # Interpolación suave
        )
        plt.colorbar(self.img)
        self.ax.axis('off')

    def initialize_chemicals(self):
        """Inicializar con manchas aleatorias"""
        U = np.ones((self.size, self.size))
        V = np.zeros((self.size, self.size))
        
        # Añadir 100 manchas aleatorias
        for _ in range(100):
            x = np.random.randint(10, self.size-10)
            y = np.random.randint(10, self.size-10)
            r = np.random.randint(5, 10)  # Tamaño aleatorio de las manchas
            V[x-r:x+r, y-r:y+r] = 0.8 + 0.2 * np.random.randn(2*r, 2*r)
        
        return U, V

    def laplacian(self, Z):
        """Stencil de 5 puntos para la difusión"""
        return (np.roll(Z, (0, -1), (0, 1)) + 
                np.roll(Z, (0, 1), (0, 1)) +
                np.roll(Z, (-1, 0), (0, 1)) + 
                np.roll(Z, (1, 0), (0, 1)) - 
                4 * Z)

    def update(self):
        """Actualización del modelo Gray-Scott"""
        Lu = self.laplacian(self.U)
        Lv = self.laplacian(self.V)
        
        reaction = self.U * self.V**2
        dU = self.dA * Lu - reaction + self.F * (1 - self.U)
        dV = self.dB * Lv + reaction - (self.F + self.K) * self.V
        
        self.U += dU * self.dt
        self.V += dV * self.dt
        
        # Aplicar restricciones
        self.U = np.clip(self.U, 0.1, 1.0)
        self.V = np.clip(self.V, 0.0, 1.0)

    def animate(self, i):
        """Actualización de la animación"""
        for _ in range(5):  # Menos iteraciones por frame para mayor suavidad
            self.update()
        
        self.img.set_data(self.V)
        self.img.set_norm(Normalize(vmin=0, vmax=1))
        
        if i % 10 == 0:
            print(f"Frame {i}: V range = {self.V.min():.2f} - {self.V.max():.2f}")
        
        return [self.img]

    def run(self, pattern_name, save_gif=True):
        """Ejecutar la simulación para un patrón específico"""
        print(f"Simulando patrón: {pattern_name}")
        ani = animation.FuncAnimation(
            self.fig, 
            self.animate, 
            interval=50,  # 20 FPS
            blit=True, 
            save_count=5000  # Más frames para mayor duración
        )
        
        if save_gif:
            # Guardar GIF de alta calidad
            print("Guardando GIF...")
            ani.save(
                f'reaction_diffusion_{pattern_name}.gif', 
                writer='pillow', 
                fps=50, 
                dpi=300,
                bitrate=1800  # Mayor calidad
            )
            print(f"GIF guardado como 'reaction_diffusion_{pattern_name}.gif'")
        
        # Mostrar animación
        plt.show()

# Simular todos los patrones
size = 512  # Tamaño de la cuadrícula (puedes cambiarlo a 512 para mayor resolución)
for pattern_name, params in PATTERNS.items():
    sim = ReactionDiffusion(size=size, F=params["F"], K=params["K"], cmap=params["cmap"])
    sim.run(pattern_name, save_gif=True)

Simulando patrón: cerebro_coral
Frame 0: V range = 0.00 - 1.00
Frame 0: V range = 0.00 - 1.00
Guardando GIF...
Frame 0: V range = 0.00 - 1.00
Frame 0: V range = 0.00 - 1.00
Frame 10: V range = 0.00 - 0.89
Frame 20: V range = 0.00 - 0.73
Frame 30: V range = 0.00 - 0.63
Frame 40: V range = 0.00 - 0.57
Frame 50: V range = 0.00 - 0.53
Frame 60: V range = 0.00 - 0.51
Frame 70: V range = 0.00 - 0.49
Frame 80: V range = 0.00 - 0.48
Frame 90: V range = 0.00 - 0.47
Frame 100: V range = 0.00 - 0.47
Frame 110: V range = 0.00 - 0.47
Frame 120: V range = 0.00 - 0.47
Frame 130: V range = 0.00 - 0.47
Frame 140: V range = 0.00 - 0.47
Frame 150: V range = 0.00 - 0.47
Frame 160: V range = 0.00 - 0.47
Frame 170: V range = 0.00 - 0.47
Frame 180: V range = 0.00 - 0.47
Frame 190: V range = 0.00 - 0.47
Frame 200: V range = 0.00 - 0.47
Frame 210: V range = 0.00 - 0.47
Frame 220: V range = 0.00 - 0.47
Frame 230: V range = 0.00 - 0.47
Frame 240: V range = 0.00 - 0.47
Frame 250: V range = 0.00 - 0.47
Frame 260: 